In [1]:
from darts import TimeSeries
from darts.utils.timeseries_generation import linear_timeseries
from darts.utils.missing_values import fill_missing_values

frequency = '10min'

# Assuming you have a pandas DataFrame with a column named 'timestamp' and a column named 'power'
# and you want to split the data into 80% training and 20% testing

# Convert your data to a Darts TimeSeries object
use_columns = ['Wind direction (°)','Nacelle position (°)','blade_angle','Rear bearing temperature (°C)',
                'Rotor speed (RPM)','Generator RPM (RPM)','Nacelle ambient temperature (°C)',
                'Front bearing temperature (°C)','Tower Acceleration X (mm/ss)','Wind speed (m/s)','Tower Acceleration y (mm/ss)','Metal particle count counter']

series = TimeSeries.from_csv('input_data.csv', time_col='# Date and time',value_cols ='Power (kW)', fill_missing_dates=True, freq=frequency)

In [2]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.utils.timeseries_generation import (
    gaussian_timeseries,
    linear_timeseries,
    sine_timeseries,
)
from darts.models import (
    RNNModel,
    TCNModel,
    TransformerModel,
    NBEATSModel,
    BlockRNNModel,
    VARIMA,
)
from darts.metrics import mape, smape, mae
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
import logging

logging.disable(logging.CRITICAL)

import warnings

warnings.filterwarnings("ignore")
torch.manual_seed(1)
np.random.seed(1)

In [3]:
stack_covariates = ['Wind direction (°)',
       'Nacelle position (°)','Front bearing temperature (°C)',
       'Rear bearing temperature (°C)', 'Nacelle ambient temperature (°C)',
       'Rotor speed (RPM)', 'Generator RPM (RPM)',
       'Metal particle count counter', 'Tower Acceleration X (mm/ss)',
       'Tower Acceleration y (mm/ss)', 'blade_angle',
       'Wind direction (°)_lagged_1', 'Wind direction (°)_lagged_2',
       'Wind direction (°)_lagged_3', 'Wind direction (°)_lagged_4',
       'Wind direction (°)_lagged_5', 'Rear bearing temperature (°C)_lagged_1',
       'Rear bearing temperature (°C)_lagged_2',
       'Rear bearing temperature (°C)_lagged_3',
       'Rear bearing temperature (°C)_lagged_4',
       'Rear bearing temperature (°C)_lagged_5', 'Rotor speed (RPM)_lagged_1',
       'Rotor speed (RPM)_lagged_2', 'Rotor speed (RPM)_lagged_3',
       'Rotor speed (RPM)_lagged_4', 'Rotor speed (RPM)_lagged_5',
       'Generator RPM (RPM)_lagged_1', 'Generator RPM (RPM)_lagged_2',
       'Generator RPM (RPM)_lagged_3', 'Generator RPM (RPM)_lagged_4',
       'Generator RPM (RPM)_lagged_5',
       'Front bearing temperature (°C)_lagged_1',
       'Front bearing temperature (°C)_lagged_2',
       'Front bearing temperature (°C)_lagged_3',
       'Front bearing temperature (°C)_lagged_4',
       'Front bearing temperature (°C)_lagged_5',
       'Tower Acceleration X (mm/ss)_lagged_1',
       'Tower Acceleration X (mm/ss)_lagged_2',
       'Tower Acceleration X (mm/ss)_lagged_3',
       'Tower Acceleration X (mm/ss)_lagged_4',
       'Tower Acceleration X (mm/ss)_lagged_5', 'Wind speed (m/s)_lagged_1',
       'Wind speed (m/s)_lagged_2', 'Wind speed (m/s)_lagged_3',
       'Wind speed (m/s)_lagged_4', 'Wind speed (m/s)_lagged_5',
       'Tower Acceleration y (mm/ss)_lagged_1',
       'Tower Acceleration y (mm/ss)_lagged_2',
       'Tower Acceleration y (mm/ss)_lagged_3',
       'Tower Acceleration y (mm/ss)_lagged_4',
       'Tower Acceleration y (mm/ss)_lagged_5',
       'Metal particle count counter_lagged_1',
       'Metal particle count counter_lagged_2',
       'Metal particle count counter_lagged_3',
       'Metal particle count counter_lagged_4',
       'Metal particle count counter_lagged_5', 'blade_angle_diff',
       'Nacelle ambient temperature (°C)_diff',
       'Tower Acceleration X (mm/ss)_diff',
       'Tower Acceleration y (mm/ss)_diff']

covar_series= TimeSeries.from_csv('input_data.csv', time_col='# Date and time',value_cols ='Wind speed (m/s)', fill_missing_dates=True, freq=frequency)


for covariate in stack_covariates:
    covariate = covariate.strip()
    series = f'{covariate}_series'
    print(series)
    series = TimeSeries.from_csv('input_data.csv', time_col='# Date and time',value_cols =covariate, fill_missing_dates=True, freq=frequency)
    print('done')
    covar_series = covar_series.stack(series)


Wind direction (°)_series
done
Nacelle position (°)_series
done
Front bearing temperature (°C)_series
done
Rear bearing temperature (°C)_series
done
Nacelle ambient temperature (°C)_series
done
Rotor speed (RPM)_series
done
Generator RPM (RPM)_series
done
Metal particle count counter_series
done
Tower Acceleration X (mm/ss)_series
done
Tower Acceleration y (mm/ss)_series
done
blade_angle_series
done
Wind direction (°)_lagged_1_series
done
Wind direction (°)_lagged_2_series
done
Wind direction (°)_lagged_3_series
done
Wind direction (°)_lagged_4_series
done
Wind direction (°)_lagged_5_series
done
Rear bearing temperature (°C)_lagged_1_series
done
Rear bearing temperature (°C)_lagged_2_series
done
Rear bearing temperature (°C)_lagged_3_series
done
Rear bearing temperature (°C)_lagged_4_series
done
Rear bearing temperature (°C)_lagged_5_series
done
Rotor speed (RPM)_lagged_1_series
done
Rotor speed (RPM)_lagged_2_series
done
Rotor speed (RPM)_lagged_3_series
done
Rotor speed (RPM)_lagged_

In [4]:
covar_series.to_csv('covar.csv')

In [ ]:
percent_data = 0.02
# Split the time series into training and testing sets
train_size = int(len(series) * percent_data)  
train, test = series[:train_size], series[train_size:]
covar_train, covar_test = covar_series[:train_size], series[train_size:]


In [9]:
my_model = TransformerModel(
    input_chunk_length=180,
    output_chunk_length=6,
    batch_size=540,
    n_epochs=20,
    model_name="wind_transformer",
    nhead=8,
num_encoder_layers=10, num_decoder_layers=10

)


In [10]:
# fit using multiple (two) target series
my_model.fit(train,
          past_covariates=covar_train,)

Training: 0it [00:00, ?it/s]

RuntimeError: [enforce fail at C:\b\abs_bao0hdcrdh\croot\pytorch_1675190257512\work\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 49766400 bytes.